In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_num.csv")
df.head()

In [ ]:
df = df[['NU_NOTA_CH',
        'NU_NOTA_CN',
        'NU_NOTA_MT',
        'NU_NOTA_LC',
        'NU_NOTA_REDACAO',
        'EST_IDADE',
        'EST_RENDA_PER_CAP',
        'EST_CELULAR_PER_CAP',
        'EST_COMP_PER_CAP',
        'EST_VEICULO_PER_CAP',
        'EST_ELE_DOM_PER_CAP'
        ]]

df.head()

In [ ]:
colunas = ['NU_NOTA_CH',
           'NU_NOTA_CN',
           'NU_NOTA_MT',
           'NU_NOTA_LC',
           'NU_NOTA_REDACAO',
           'EST_IDADE',
           'EST_RENDA_PER_CAP',
           'EST_CELULAR_PER_CAP',
           'EST_COMP_PER_CAP',
           'EST_VEICULO_PER_CAP',
           'EST_ELE_DOM_PER_CAP'
          ]

Matriz de correlação

In [ ]:
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de Correlação - Variáveis Padronizadas")
plt.show()

In [ ]:
scaler = StandardScaler()
df_padronizado = scaler.fit_transform(df)

Matriz de Covariância

In [ ]:
cov_scaled = np.cov(df_padronizado, rowvar=False)
cov_df = pd.DataFrame(cov_scaled, index=colunas, columns=colunas)

# Visualiza
plt.figure(figsize=(6,5))
sns.heatmap(cov_df, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title("Matriz de Covariância - Variáveis Padronizadas")
plt.show()

Implementação do PCA

In [ ]:
pca = PCA()
df_pca = pca.fit_transform(df_padronizado)

In [ ]:
var_exp = pca.explained_variance_ratio_
var_exp_acum = np.cumsum(var_exp)

Tabela dos Autovalores e Variância Explicada dos Componentes Principais

In [ ]:
# ======== Tabela 1: Autovalores (Eigenvalues) =========
eigenvalues = pca.explained_variance_
diff = np.append(np.diff(eigenvalues), np.nan)
prop = pca.explained_variance_ratio_
cum = np.cumsum(prop)

tabela1 = pd.DataFrame({
    'Component': range(1, len(eigenvalues)+1),
    'Eigenvalue': eigenvalues,
    'Difference': diff,
    'Proportion': prop,
    'Cumulative': cum
})

print("\n=== Eigenvalues (Autovalores) ===")
print(tabela1.round(4))

Matriz de Cargas (Loadings) da Análise de Componentes Principais

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    index=colunas,
    columns=[f'PC{i+1}' for i in range(len(eigenvalues))]
)

print("\n=== Loadings (Autovetores / Pesos das Variáveis) ===")
print(loadings.round(3))

In [ ]:
plt.figure(figsize=(7, 5))
sns.heatmap(loadings, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Heatmap - Cargas das Variáveis (Loadings) nos Componentes Principais")
plt.xlabel("Componentes Principais")
plt.ylabel("Variáveis Originais")
plt.show()

Scree Plot (PCA)

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(range(1, len(eigenvalues)+1), eigenvalues, marker='o')
plt.title("Scree Plot")
plt.xlabel("Componente Principal")
plt.ylabel("Variância Explicada")
plt.grid(True)
plt.show()

Biplot

In [ ]:
plt.figure(figsize=(7,6))
for i, var in enumerate(colunas):
    plt.arrow(0, 0, loadings.PC1[i], loadings.PC2[i],
              head_width=0.02, color='blue')
    plt.text(loadings.PC1[i]*1.15, loadings.PC2[i]*1.15, var, color='black')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Biplot - Cargas das Variáveis nos Dois Primeiros Componentes')
plt.grid(True)
plt.show()

Gráfico da Dispersão das Variáveis Originais com os Eixos das Componentes Principais

In [ ]:
mean = df_padronizado.mean(axis=0)
vectors = pca.components_
explained = pca.explained_variance_

# --- Plotar os pontos ---
plt.figure(figsize=(8, 8))
plt.scatter(df_padronizado[:, 0], df_padronizado[:, 1], alpha=0.6, label="Dados originais")

# --- Plotar o ponto médio ---
plt.scatter(mean[0], mean[1], color='red', s=100, label='Média (x̄,ȳ)')

# --- Plotar as retas das componentes principais (eixos Y1 e Y2) ---
for i, (vec, var) in enumerate(zip(vectors[:2], explained[:2])):
    # Escala para deixar visível no gráfico
    scale = 3 * np.sqrt(var)
    x_line = np.array([-scale, scale])
    y_line = vec[1]/vec[0] * x_line
    plt.plot(mean[0] + x_line, mean[1] + y_line,
             linestyle='--' if i==0 else '-',
             color='black', linewidth=2,
             label=f'Eixo PC{i+1}')

# --- Personalização ---
plt.axhline(mean[1], color='gray', linestyle=':')
plt.axvline(mean[0], color='gray', linestyle=':')
plt.xlabel("X1")
plt.ylabel("X2")
plt.title("Dispersão de X1 e X2 com Eixos das Componentes Principais (Y1 e Y2)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)
plt.axis('equal')
plt.show()

DF com os novos componentes

In [ ]:
pca_df = pd.DataFrame(df_pca[:, :3], columns=['PC1', 'PC2', 'PC3'])
print("\nComponentes principais (primeiras linhas):")
pca_df.head()